In [1]:
from google.cloud import bigquery
import operator
import networkx as nx
from nltk.util import ngrams
from nltk import FreqDist
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import os
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import csv 
import sys #used for passing in the argument
import pickle

In [2]:
from google.cloud import bigquery
G = nx.Graph()
client = bigquery.Client.from_service_account_json(
        '../../../Downloads/MyFirstProject-e624aa75f64b.json')
client.project

'indigo-bazaar-255505'

In [3]:
dataset_id = 'my_dataset'

dataset_ref = client.dataset(dataset_id)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_ref)
# Specify the geographic location where the dataset should reside.
dataset.location = 'US'

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.AlreadyExists if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # API request

In [5]:
file_name = "data.csv" #filename is argument 1
with open(file_name, 'r') as f:  #opens PW file
    reader = csv.reader(f)
    data = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists


KeyboardInterrupt: 

In [3]:
from gensim.models import FastText
from gensim.models import KeyedVectors

fasttext_model = FastText.load("../../../Downloads/fast_text_model")

In [6]:
tags_similar = fasttext_model.wv.most_similar(["golang"])
tags_similar

[('clang', 0.6894737482070923),
 ('rust', 0.6086438298225403),
 ('erlang', 0.6015114188194275),
 ('julia-lang', 0.5971020460128784),
 ('llvm', 0.5818730592727661),
 ('d', 0.5587214827537537),
 ('gcc', 0.5034722089767456),
 ('c++11', 0.4999593496322632),
 ('ocaml', 0.49894481897354126),
 ('boost', 0.497931569814682)]

In [7]:
word2vec_model = KeyedVectors.load("../../../Downloads/word2vec_model")

In [12]:
tags_similar = word2vec_model.wv.most_similar(["go", "interface", "struct", "pointers"])
tags_similar

[('sizeof', 0.7168252468109131),
 ('function-pointers', 0.7156633734703064),
 ('unions', 0.6719181537628174),
 ('typedef', 0.6702909469604492),
 ('const', 0.6694769859313965),
 ('dynamic-memory-allocation', 0.6656872034072876),
 ('d', 0.6528905630111694),
 ('ada', 0.634309709072113),
 ('structure', 0.6263110637664795),
 ('pass-by-reference', 0.6256006956100464)]

In [55]:
for word, score in tags_similar:
    query = """
        SELECT  * FROM `bigquery-public-data.stackoverflow.posts_questions` 
        where id in (SELECT related_post_id FROM `bigquery-public-data.stackoverflow.post_links` group by related_post_id)
        and tags like @a order by score desc limit 1 
        """
    query_params = [
    bigquery.ScalarQueryParameter(
        'a', 'STRING', "%"+word+"%")
    ]
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    query_job = client.query(query, job_config=job_config)
    results = query_job.result()

    for row in results:
        print(row.tags, row.score, row.title)





producer-consumer
java|multithreading|producer-consumer|blockingqueue 60 Blocking queue and multi-threaded consumer, how to know when to stop
threadpool
c#|.net|task-parallel-library|async-await|threadpool 269 If my interface must return Task what is the best way to have a no-operation implementation?
semaphore
operating-system|mutex|semaphore|glossary 626 Difference between binary semaphore and mutex
thread-safety
c#|wcf|concurrency|dictionary|thread-safety 678 Collection was modified; enumeration operation may not execute
concurrency
java|multithreading|concurrency|thread-local|thread-confinement 766 When and how should I use a ThreadLocal variable?
nonblocking
python|io|subprocess|nonblocking 414 Non-blocking read on a subprocess.PIPE in python
ipc
regex|zipcode|postal-code 164 What is the ultimate postal code and zip regex?
blocking
python|io|subprocess|nonblocking 414 Non-blocking read on a subprocess.PIPE in python
deadlock
multithreading|pthreads|deadlock|livelock 228 What's the

In [50]:
# how do u find out the number of the views of the question? find out the number of questions with each linked questions, that should tell the important of the question

In [51]:
# how many questions related with each linked question sorted by the count. 
# SELECT count(*) as count, related_post_id FROM [bigquery-public-data:stackoverflow.post_links] group by related_post_id order by count desc

In [ ]:
# But we want to find such questions within each cluster  sorted by the tags. 

# how do u find number of linked questions with each tag. 
query = """
        SELECT  * FROM `bigquery-public-data.stackoverflow.posts_questions` 
        where id in (SELECT related_post_id FROM `bigquery-public-data.stackoverflow.post_links` group by related_post_id)
        and tags like @a order by score desc limit 1 
        """
query_params = [
bigquery.ScalarQueryParameter(
    'a', 'STRING', "%"+word+"%")
]
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params
query_job = client.query(query, job_config=job_config)
results = query_job.result()

for row in results:
    print(row.tags, row.score, row.title)


In [12]:
# This prints the tags of the linked questions together with their count sorted by count desc. We need to find out which tag has the most count. This way we can atleast do another sort and figure out the importance. 

query = """
SELECT
  d.related_post_id as id, a.tags, d.c as final_count, a.score as score
FROM
  `bigquery-public-data.stackoverflow.posts_questions` AS a
INNER JOIN (
  SELECT
    COUNT(*) AS c,
    related_post_id
  FROM
    `bigquery-public-data.stackoverflow.post_links` AS b
  GROUP BY
    related_post_id ) As d
ON
  a.id = d.related_post_id

order by final_count desc
"""
query_job = client.query(query)
results = query_job.result()
file_name = "related_post_id_tags_count_linked_questions.csv"
if not os.path.exists(file_name):
    results = query_job.result()
    with open(file_name, 'w') as myfile:
        wr = csv.writer(myfile)
        for row in results:
            wr.writerow(row.tags.split("|") + [row.final_count] + [row.score] + [row.id])
    

In [60]:
file


<_io.BufferedWriter name='tags_count_linked_questions.csv'>

In [15]:
tag_count_hash = {}
with open(file_name, 'r') as myfile:
    for line in myfile:
        *words, count, score, id_ = line.split(",")
        for word in words:
            if word not in tag_count_hash:
                tag_count_hash[word] = {'count': 1, 'id': id_.strip(), 'score': score}
            else:
                tag_count_hash[word]['count'] += 1
                

In [16]:
tag_count_hash  

{'java': {'count': 169497, 'id': '218384', 'score': '211'},
 'nullpointerexception': {'count': 1174, 'id': '218384', 'score': '211'},
 'r': {'count': 33392, 'id': '5963269', 'score': '2325'},
 'r-faq': {'count': 163, 'id': '5963269', 'score': '2325'},
 'javascript': {'count': 145579, 'id': '14220321', 'score': '4156'},
 'ajax': {'count': 16538, 'id': '14220321', 'score': '4156'},
 'asynchronous': {'count': 3942, 'id': '14220321', 'score': '4156'},
 'ecmascript-6': {'count': 1901, 'id': '14220321', 'score': '4156'},
 'ecmascript-2017': {'count': 38, 'id': '14220321', 'score': '4156'},
 'php': {'count': 103660, 'id': '60174', 'score': '2782'},
 'mysql': {'count': 42093, 'id': '60174', 'score': '2782'},
 'sql': {'count': 38054, 'id': '60174', 'score': '2782'},
 'security': {'count': 6312, 'id': '60174', 'score': '2782'},
 'sql-injection': {'count': 487, 'id': '60174', 'score': '2782'},
 'c#': {'count': 149486, 'id': '4660142', 'score': '1880'},
 '.net': {'count': 45586, 'id': '4660142', '

In [17]:
import json

with open('tag_count_score_hash.json', 'w') as fp:
    json.dump(tag_count_hash, fp)

In [18]:
import csv
 
myFile = open('tag_count_hash_sorted.csv', 'w')
writer = csv.writer(myFile)


for key in sorted(tag_count_hash.keys(), key=lambda x: tag_count_hash[x]['count'],  reverse=True):
    writer.writerow([key, tag_count_hash[key]])
print("Writing complete")

Writing complete


In [23]:
import csv
 
myFile = open('tag_score_hash_sorted.csv', 'w')
writer = csv.writer(myFile)


for key in sorted(tag_count_hash.keys(), key=lambda x: int(tag_count_hash[x]['score']),  reverse=True):
    writer.writerow([key, tag_count_hash[key]])
print("Writing complete")

Writing complete


In [23]:
tags_similar = word2vec_model.wv.most_similar(["operating-system"], topn=30)
for tag, similarity in tags_similar:
    print(tag, similarity, tag_count_hash[tag])

NameError: name 'tag_count_hash' is not defined

In [24]:
tag_count_hash

{'java': {'count': 169497, 'id': '218384', 'score': '211'},
 'nullpointerexception': {'count': 1174, 'id': '218384', 'score': '211'},
 'r': {'count': 33392, 'id': '5963269', 'score': '2325'},
 'r-faq': {'count': 163, 'id': '5963269', 'score': '2325'},
 'javascript': {'count': 145579, 'id': '14220321', 'score': '4156'},
 'ajax': {'count': 16538, 'id': '14220321', 'score': '4156'},
 'asynchronous': {'count': 3942, 'id': '14220321', 'score': '4156'},
 'ecmascript-6': {'count': 1901, 'id': '14220321', 'score': '4156'},
 'ecmascript-2017': {'count': 38, 'id': '14220321', 'score': '4156'},
 'php': {'count': 103660, 'id': '60174', 'score': '2782'},
 'mysql': {'count': 42093, 'id': '60174', 'score': '2782'},
 'sql': {'count': 38054, 'id': '60174', 'score': '2782'},
 'security': {'count': 6312, 'id': '60174', 'score': '2782'},
 'sql-injection': {'count': 487, 'id': '60174', 'score': '2782'},
 'c#': {'count': 149486, 'id': '4660142', 'score': '1880'},
 '.net': {'count': 45586, 'id': '4660142', '

In [33]:
# how do u find which term is important and is important to show to the user. 
# show one question from each cluster. But i think then among all the terms in the cluster find the most important question. 
# for that tag which question has the most score? 

# get all the tags and store them in a file.


In [46]:
# This prints the tags of the linked questions together with their count sorted by count desc. We need to find out which tag has the most count. This way we can atleast do another sort and figure out the importance. 

query = """
SELECT tag_name, count FROM `bigquery-public-data.stackoverflow.tags` order by count desc
"""
query_job = client.query(query)
results = query_job.result()
file_name = "tags.csv"
if not os.path.exists(file_name):
    results = query_job.result()
    with open(file_name, "w") as text_file:
        wr = csv.writer(text_file)
        for row in results:
            wr.writerow([row.count] + [row.tag_name])
    
            
    

In [50]:
tags = ["numpy"]
tags_similar = fasttext_model.wv.most_similar(tags, topn = 20)
tags += [x[0] for x in tags_similar]
tags

['numpy',
 'sympy',
 'scikit-learn',
 'scipy',
 'sparse-matrix',
 'tensorflow',
 'vectorization',
 'python-3.x',
 'cython',
 'curve-fitting',
 'matrix-multiplication',
 'pandas',
 'python-2.7',
 'keras',
 'python-3.5',
 'list-comprehension',
 'python-3.3',
 'python-3.6',
 'python-2.x',
 'matplotlib',
 'linear-regression']

In [41]:
for tag in tags:
    query = """
    select id, title,score, tags from `bigquery-public-data.stackoverflow.posts_questions` where tags like @a and id in (SELECT related_post_id FROM `bigquery-public-data.stackoverflow.post_links` group by related_post_id) order by score desc limit 10
    """
    query_params = [
        bigquery.ScalarQueryParameter(
            'a', 'STRING', "%"+tag+"%")
        ]
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    query_job = client.query(query, job_config=job_config)
    results = query_job.result()
    for row in results:
        print(row.id, row.title, row.score, row.tags)

            


406294 LEFT JOIN vs. LEFT OUTER JOIN in SQL Server 1185 sql-server|tsql|left-join|outer-join
4715677 Difference between left join and right join in SQL Server 196 sql|sql-server-2005|join|left-join|right-join
267488 Linq to Sql: Multiple left outer joins 152 vb.net|linq|linq-to-sql|tsql|left-join
2763206 Deleting rows with MySQL LEFT JOIN 148 mysql|left-join|delete-row|sql-delete
4752455 Left Join With Where Clause 105 sql|left-join|where-clause
5307731 LINQ to SQL: Multiple joins ON multiple Columns. Is this possible? 105 c#|linq-to-sql|left-join|multiple-columns
19363481 Select rows which are not present in other table 104 sql|postgresql|null|left-join|exists
15626493 LEFT JOIN only first row 88 mysql|join|left-join
1509692 Rails ActiveRecord :joins with LEFT JOIN instead of INNER JOIN 66 ruby-on-rails|activerecord|join|left-join|inner-join
16470942 How to use mysql JOIN without ON condition? 61 mysql|join|left-join|inner-join|right-join
38549 What is the difference between "INNER JO

354070 SQL join: where clause vs. on clause 483 sql|join|where-clause|on-clause
2905292 WHERE vs HAVING 206 mysql|sql|where-clause|having-clause
4752455 Left Join With Where Clause 105 sql|left-join|where-clause
1219909 MySQL join with where clause 104 mysql|join|where-clause
958627 MySQL - ORDER BY values within IN() 81 mysql|sql-order-by|where-clause
11436469 Does order of where clauses matter in SQL? 78 sql|performance|where-clause
2101540 "Or" equivalent in Linq Where() lambda expression 76 .net|linq|where-clause
4047484 SELECTING with multiple WHERE conditions on same column 49 mysql|sql|aggregate-functions|where-clause
848415 Dynamic WHERE clause in LINQ 48 c#|linq|dynamic|where-clause
1129923 Is a JOIN faster than a WHERE? 47 sql|performance|join|where-clause
49925 What is the difference between UNION and UNION ALL? 1078 sql|union|union-all
346536 Difference between a Structure and a Union 345 c|struct|unions
252552 Why do we need C Unions? 200 c|unions
905379 What is the differ

5859391 Create a temporary table in a SELECT statement without a separate CREATE TABLE 419 mysql|select|temp-tables|create-table|derived-table
4098008 Create table in SQLite only if it doesn't exist already 177 sqlite|create-table|database-table
1766046 PostgreSQL create table if not exists 90 postgresql|create-table|not-exists|database-table
4692690 Is it possible to roll back CREATE TABLE and ALTER TABLE statements in major SQL databases? 86 sql|transactions|ddl|create-table
9826833 create table in postgreSQL 48 postgresql|create-table
2415855 Copy a MySQL table including indexes 39 mysql|indexing|create-table
11545189 SQLite Foreign Key 36 android|sql|sqlite|foreign-key-relationship|create-table
22637733 MySQL: Error Code: 1118 Row size too large (> 8126). Changing some columns to TEXT or BLOB 35 mysql|sql|create-table
14285854 Cannot create a new table after "DROP SCHEMA public" 30 database|postgresql|database-schema|create-table|sql-drop
13289107 What does regclass signify in Post

In [13]:
query = """
     
    """
query_job = client.query(query)
results = query_job.result()
file_name = "questions_answers_user_id.csv"
if not os.path.exists(file_name):
    results = query_job.result()
    with open(file_name, "w") as text_file:
        wr = csv.writer(text_file)
        for row in results:
            wr.writerow([row.aid, row.qid, row.auid, row.quid])
    
            
    


In [11]:
# get all the questions with the questions that link to it. 
# get all the related_post_id because we need their titles as text. 

#SELECT distinct related_post_id FROM [bigquery-public-data:stackoverflow.post_links] group by related_post_id

# then iterate over the related_post_id and get all the questions id. and then for each question id find its, title, body, score. 
# similarly for each related_post_id

query = """
    select id, title, body, tags from `bigquery-public-data.stackoverflow.posts_questions` where id in (SELECT distinct related_post_id FROM `bigquery-public-data.stackoverflow.post_links` group by related_post_id)
    """
query_job = client.query(query)
results = query_job.result()
for row in results:
    file_name = "related_post_id_title_body_tags.csv"
    if not os.path.exists(file_name):
        results = query_job.result()
        with open(file_name, "w") as text_file:
            wr = csv.writer(text_file)
            for row in results:
                wr.writerow([row.id] + [row.title] + [row.body] + [row.tags])

In [7]:
query = """
    SELECT  related_post_id, post_id FROM `bigquery-public-data.stackoverflow.post_links`
    """
query_job = client.query(query)
results = query_job.result()
h = {}
for row in results:
    if row.related_post_id in h:
        h[row.related_post_id].append(row.post_id)
    else:
        h[row.related_post_id] = [row.post_id]


In [21]:

with open("related_post_id_post_id" + '.pkl', 'wb') as f:
        pickle.dump(h, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open("related_post_id_post_id" + '.pkl', 'rb') as f:
    h = pickle.load(f)

In [4]:
len(h)

1780590

In [ ]:
# for each key build a document of all the answers and then find if the tags and document topics make sense. I need answers as well. 
import urllib.request
from __future__ import print_function
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, ConceptsOptions


natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_api_key="673L3LYkJ3Htp50o31IbSzRI2cC4UfsEQMzly_3cJUT1",
    url= "https://gateway-wdc.watsonplatform.net/natural-language-understanding/api"
)

for related_post_id in h:
    try:
        ids = [related_post_id] + h[related_post_id]
        query = """
        select title, body from `bigquery-public-data.stackoverflow.posts_questions` where id in UNNEST(@a)
        """
        query_params = [
         bigquery.ArrayQueryParameter(
            'a', 'INT64', [related_post_id] + h[related_post_id] )    
        ]
        job_config = bigquery.QueryJobConfig()
        job_config.query_parameters = query_params
        query_job = client.query(query, job_config=job_config)
        results = query_job.result()
        query = """
        select body from `bigquery-public-data.stackoverflow.posts_answers` where parent_id in UNNEST(@b)
        """
        answers = ""

        query_params = [
         bigquery.ArrayQueryParameter(
            'b', 'INT64', ids )    
        ]
        job_config = bigquery.QueryJobConfig()
        job_config.query_parameters = query_params
        query_job = client.query(query, job_config=job_config)
        results = query_job.result()
        for row in results:
            answers += row.body
        response = natural_language_understanding.analyze(
          text = answers,
          features=Features(
            keywords=KeywordsOptions(
              limit=20)))
        file_name = "answers/" + str(related_post_id) + ".csv"
        wr = csv.writer(open(file_name, "w"))
        wr.writerow([related_post_id] + [json.dumps(response, indent=2)])
        
        with open("answers/" + str(related_post_id) + ".txt", "w") as text_file:
            text_file.write(answers)
    except:
        pass


In [36]:
h[6799172]

[11174691,
 14144279,
 23873731,
 13159337,
 15595488,
 27925313,
 30511637,
 36794667,
 35092596,
 5956534,
 19317306,
 32880295,
 28030442,
 20130446,
 26043924,
 42448373,
 45760187,
 34753155,
 28030442]

In [12]:
import csv
documents = []


with open('result.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip header
    for row in reader:
        documents.append(row[0])
        

In [49]:
from gensim import corpora
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(text) for text in text_data]


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [15]:
documents[0]

'<p><strong>You are a victim of <a href="//en.wikipedia.org/wiki/Branch_predictor" rel="noreferrer">branch prediction</a> fail.</strong></p>\n\n<hr>\n\n<h2>What is Branch Prediction?</h2>\n\n<p>Consider a railroad junction:</p>\n\n<p><a href="//commons.wikimedia.org/wiki/File:Entroncamento_do_Transpraia.JPG" rel="noreferrer"><img src="https://i.stack.imgur.com/muxnt.jpg" alt="Licensed Image"></a>\n<sub><a href="//commons.wikimedia.org/wiki/File:Entroncamento_do_Transpraia.JPG" rel="noreferrer">Image</a> by Mecanismo, via Wikimedia Commons. Used under the <a href="//creativecommons.org/licenses/by-sa/3.0/deed.en" rel="noreferrer">CC-By-SA 3.0</a> license.</sub></p>\n\n<p>Now for the sake of argument, suppose this is back in the 1800s - before long distance or radio communication.</p>\n\n<p>You are the operator of a junction and you hear a train coming. You have no idea which way it is supposed to go. You stop the train to ask the driver which direction they want. And then you set the sw

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [43]:
documents_conv= [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(documents)]

In [44]:
model = Doc2Vec(documents_conv, vector_size=5, window=2, min_count=1, workers=4)

In [45]:
model.docvecs[0]

array([-1.8061583 , -1.7755439 ,  1.8701165 , -1.5083413 ,  0.52778304],
      dtype=float32)

In [24]:
from gensim.test.utils import get_tmpfile

In [25]:
fname = get_tmpfile("my_doc2vec_model")
model.save(fname)
model = Doc2Vec.load(fname)

In [47]:
model.most_similar("branch")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('<code>&lt;password&gt;</code>', 0.9999859929084778),
 ('self.textField.rightViewMode', 0.9993543028831482),
 ('{|i|};', 0.9981523752212524),
 ('rel="noreferrer"><code>opcache_get_status()</code></a>:</h3>',
  0.9973191022872925),
 ('SendCommand', 0.997104287147522),
 ('image.', 0.997023344039917),
 ('throttle', 0.9970079660415649),
 ('RubyMotion', 0.9968304634094238),
 ('<pre><code>years_dict', 0.996543824672699),
 ('name="search"/&gt;', 0.9964763522148132)]

In [41]:
documents_conv[0]

TaggedDocument(words=['<p><strong>You', 'are', 'a', 'victim', 'of', '<a', 'href="//en.wikipedia.org/wiki/Branch_predictor"', 'rel="noreferrer">branch', 'prediction</a>', 'fail.</strong></p>', '<hr>', '<h2>What', 'is', 'Branch', 'Prediction?</h2>', '<p>Consider', 'a', 'railroad', 'junction:</p>', '<p><a', 'href="//commons.wikimedia.org/wiki/File:Entroncamento_do_Transpraia.JPG"', 'rel="noreferrer"><img', 'src="https://i.stack.imgur.com/muxnt.jpg"', 'alt="Licensed', 'Image"></a>', '<sub><a', 'href="//commons.wikimedia.org/wiki/File:Entroncamento_do_Transpraia.JPG"', 'rel="noreferrer">Image</a>', 'by', 'Mecanismo,', 'via', 'Wikimedia', 'Commons.', 'Used', 'under', 'the', '<a', 'href="//creativecommons.org/licenses/by-sa/3.0/deed.en"', 'rel="noreferrer">CC-By-SA', '3.0</a>', 'license.</sub></p>', '<p>Now', 'for', 'the', 'sake', 'of', 'argument,', 'suppose', 'this', 'is', 'back', 'in', 'the', '1800s', '-', 'before', 'long', 'distance', 'or', 'radio', 'communication.</p>', '<p>You', 'are', '

In [7]:
query = """
    SELECT sum(score) as total_score FROM `bigquery-public-data.stackoverflow.posts_questions`
    """
query_job = client.query(query)
results = query_job.result()

In [8]:
total_score = 0
for row in results:
    total_score = row.total_score

In [9]:
total_score

30897197

In [14]:
query = """
    SELECT
      COUNT(DISTINCT related_post_id) as number
    FROM
      `bigquery-public-data.stackoverflow.post_links`
    """

In [18]:
query_job = client.query(query)
results = query_job.result()

In [19]:
number_of_linked_questions = 0
for row in results:
    number_of_linked_questions = row.number
    print(row.number)

1780590


In [20]:
number_of_linked_questions

1780590

In [21]:
# total questions = 15,930,617

In [22]:
# SELECT
#   MAX(score)
# FROM
#   [bigquery-public-data:stackoverflow.posts_questions]
# WHERE
#   id IN (
#   SELECT
#     related_post_id
#   FROM
#     [bigquery-public-data:stackoverflow.post_links]
#   GROUP BY
#     related_post_id)

In [23]:
#are all the questions with high score present in the links table ? 

In [24]:
# just use those linked questions to build the set. and the questions that link to it. 
# but even in them use only some and not all to build the graph. 

In [30]:
(1780590+3602105)/15930617

0.3378836488254033

In [197]:
# use the tags, then the question and then the answers. Or can we start just from the questions? 
query = """
    SELECT
  body
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
WHERE
  id IN (
  SELECT
    related_post_id
  FROM
    `bigquery-public-data.stackoverflow.post_links`
  GROUP BY
    related_post_id)
    and score > 100
    order by score desc
    """

In [198]:
query_job = client.query(query)
results = query_job.result()

In [199]:
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
tags_freqency_stem = Counter()
tags_doc_freq = Counter()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
for row in results:
    question = row.body.lower()
    question_text = BeautifulSoup(question, "lxml").get_text()
    tokens = tokenizer.tokenize(question_text)
    filtered_sentence = [w for w in tokens if not w in stop_words]
    
    filtered_stems = [stemmer.stem(plural) for plural in filtered_sentence]
    filtered_stems_set = set(filtered_stems)
    term_frequency = Counter(filtered_stems)
    tags_doc_freq += Counter(filtered_stems_set)
    tags_freqency_stem += (term_frequency)
    
print(tags_freqency_stem, tags_doc_freq)

Counter({'use': 21438, '1': 13381, '0': 12626, 'file': 10671, 'like': 10157, 'get': 9546, 'android': 8657, '2': 8339, 'code': 8131, 'want': 7893, 'string': 7877, 'way': 7830, 'class': 7651, 'work': 7398, 'tri': 7325, 'error': 7115, 'would': 6943, 'function': 6927, 'new': 6800, 'c': 6732, 'java': 6395, 'one': 6251, 'valu': 6162, 'name': 6028, '3': 5621, 'return': 5505, 'need': 5195, 'set': 5178, 'id': 5062, 'exampl': 4982, 'type': 4933, 'run': 4893, 'data': 4855, 'method': 4834, 'object': 4816, 'know': 4782, 'user': 4781, 'http': 4631, 'public': 4440, 'follow': 4414, 'creat': 4413, 'differ': 4305, 'git': 4132, 'app': 4022, 'list': 3992, 'com': 3980, 'someth': 3960, 'int': 3897, '4': 3896, 'time': 3889, 'test': 3872, 'line': 3864, 'div': 3842, 'make': 3813, 'chang': 3782, 'text': 3774, 'call': 3702, 'question': 3674, 'version': 3649, 'find': 3637, '5': 3546, 'x': 3521, 'look': 3521, 'project': 3520, 'seem': 3485, 'applic': 3416, 'system': 3284, 'var': 3259, 'view': 3247, 'server': 3158, 

In [ ]:
# html = urlopen("https://stackoverflow.com/questions/62814/difference-between-binary-semaphore-and-mutex/86021#86021")
# soup = BeautifulSoup(html, "html.parser")
# text = soup.get_text()
# stop_words = set(stopwords.words('english'))

# tokenizer = RegexpTokenizer(r'\w+')
# word_tokens = tokenizer.tokenize(text)
# filtered_sentence = [w for w in word_tokens if not w in stop_words]
# singles = [stemmer.stem(plural) for plural in filtered_sentence]
# from collections import Counter
# term_frequency = Counter(singles)

In [207]:
words =  sorted(tags_freqency_stem, key=tags_freqency_stem.get, reverse=True)

In [201]:
occuring_words =  sorted(tags_doc_freq, key=tags_doc_freq.get, reverse=True)

In [225]:
tags_freqency_stem['thread']

1328

In [224]:
words

['use',
 '1',
 '0',
 'file',
 'like',
 'get',
 'android',
 '2',
 'code',
 'want',
 'string',
 'way',
 'class',
 'work',
 'tri',
 'error',
 'would',
 'function',
 'new',
 'c',
 'java',
 'one',
 'valu',
 'name',
 '3',
 'return',
 'need',
 'set',
 'id',
 'exampl',
 'type',
 'run',
 'data',
 'method',
 'object',
 'know',
 'user',
 'http',
 'public',
 'follow',
 'creat',
 'differ',
 'git',
 'app',
 'list',
 'com',
 'someth',
 'int',
 '4',
 'time',
 'test',
 'line',
 'div',
 'make',
 'chang',
 'text',
 'call',
 'question',
 'version',
 'find',
 '5',
 'x',
 'look',
 'project',
 'seem',
 'applic',
 'system',
 'var',
 'view',
 'server',
 'also',
 'problem',
 'instal',
 'e',
 'add',
 'could',
 'see',
 'tabl',
 'possibl',
 'command',
 'python',
 'html',
 'b',
 'page',
 'script',
 'option',
 'true',
 'updat',
 'org',
 'array',
 'build',
 'select',
 'result',
 'two',
 'null',
 'first',
 'compil',
 'imag',
 'case',
 'number',
 'check',
 'window',
 'key',
 'start',
 'contain',
 'read',
 'found',
 'el

In [129]:
# we will have to somehow use the words in the question and the words in the answer for the relation and use them with the words in the tags list. 

In [206]:
occuring_words

TypeError: list indices must be integers or slices, not str

In [226]:
# for all the above questions, get the questions that link to them. maybe they casn help in building the vocab. We will have to 
# use the answers some way. 
# use the tags, then the question and then the answers. Or can we start just from the questions? 
query = """
    select body from `bigquery-public-data.stackoverflow.posts_questions`
    where id in 
    (
    select post_id from `bigquery-public-data.stackoverflow.post_links`
    where related_post_id in 
    (
    SELECT
      id
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    WHERE
      id IN (
      SELECT
        related_post_id
      FROM
        `bigquery-public-data.stackoverflow.post_links`
      GROUP BY
        related_post_id)
    and score > 100
    )
    )
    """

In [227]:
query_job = client.query(query)
results = query_job.result()

In [228]:
for row in results:
    question = row.body.lower()
    question_text = BeautifulSoup(question, "lxml").get_text()
    tokens = tokenizer.tokenize(question_text)
    filtered_sentence = [w for w in tokens if not w in stop_words]
    
    filtered_stems = [stemmer.stem(plural) for plural in filtered_sentence]
    filtered_stems_set = set(filtered_stems)
    term_frequency = Counter(filtered_stems)
    tags_doc_freq += (Counter(filtered_stems_set))
    tags_freqency_stem += (term_frequency)
    
print(tags_freqency_stem, tags_doc_freq)

ConnectionError: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))

In [229]:
words =  sorted(tags_freqency_stem, key=tags_freqency_stem.get, reverse=True)

In [233]:
words[0:100]

['0',
 '1',
 'use',
 'class',
 'android',
 'string',
 'new',
 '2',
 'file',
 'get',
 'function',
 'id',
 'code',
 'java',
 'name',
 'valu',
 'return',
 'public',
 'tri',
 'data',
 'int',
 'error',
 'div',
 'like',
 'work',
 'c',
 '3',
 'want',
 'type',
 'text',
 'com',
 'var',
 'user',
 'http',
 'method',
 'void',
 '4',
 'object',
 'would',
 'set',
 'e',
 'one',
 'way',
 'list',
 'creat',
 '5',
 'x',
 'need',
 'app',
 'php',
 'run',
 'view',
 'null',
 'input',
 'system',
 'line',
 'html',
 'select',
 'import',
 'follow',
 'true',
 'add',
 'exampl',
 'time',
 'result',
 'array',
 'n',
 'call',
 'test',
 'r',
 'b',
 'script',
 'tabl',
 'problem',
 'know',
 'button',
 'imag',
 'form',
 'p',
 'td',
 '10',
 'page',
 'question',
 'make',
 'chang',
 'main',
 'log',
 'row',
 'els',
 'privat',
 'item',
 'number',
 'option',
 'applic',
 'help',
 'org',
 'fals',
 'url',
 'first',
 'includ']

In [158]:
occuring_words =  sorted(tags_doc_freq, key=tags_doc_freq.get, reverse=True)

In [236]:
occuring_words

['use',
 'like',
 'get',
 'way',
 'want',
 'tri',
 'work',
 'code',
 'would',
 'one',
 '1',
 'file',
 'need',
 'know',
 'exampl',
 'new',
 'follow',
 '0',
 '2',
 'function',
 'differ',
 'string',
 'someth',
 'error',
 'class',
 'set',
 'creat',
 'make',
 'valu',
 'c',
 'look',
 'return',
 'seem',
 'find',
 'run',
 'question',
 'method',
 'name',
 'also',
 'type',
 '3',
 'possibl',
 'could',
 'time',
 'chang',
 'object',
 'problem',
 'see',
 'call',
 'user',
 'line',
 'data',
 'http',
 'applic',
 'list',
 'two',
 'add',
 'howev',
 'text',
 'without',
 'first',
 'java',
 '4',
 'say',
 'current',
 'project',
 'found',
 'app',
 '5',
 'read',
 'command',
 'e',
 'version',
 'server',
 'go',
 'id',
 'test',
 'case',
 'write',
 'start',
 'com',
 'updat',
 'check',
 'result',
 'answer',
 'javascript',
 'help',
 'gener',
 'public',
 'html',
 'contain',
 'thing',
 'edit',
 'solut',
 'anyon',
 'show',
 'understand',
 'page',
 'simpl',
 'var',
 'think',
 'anoth',
 'python',
 'system',
 'still',
 'i

In [238]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open('tags_doc_freq.p', 'wb') as fp:
    pickle.dump(tags_doc_freq, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [239]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open('tags_freqency_stem.p', 'wb') as fp:
    pickle.dump(tags_freqency_stem, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# when i am reading a question, this wil help me in finding which term is the most important in that question. 

In [1]:
# for understanding what the answers are, i would need to everything above for the answers. 

In [4]:
import pickle
tags_freqency_stem = pickle.load( open( "tags_freqency_stem.p", "rb" ) )
tags_doc_freq = pickle.load( open( "tags_doc_freq.p", "rb" ) )

In [5]:
tags_doc_freq

Counter({'long': 3273,
         'int': 10646,
         'string': 16717,
         'unsort': 34,
         'static_cast': 77,
         'without': 6614,
         'first': 9707,
         'time': 11021,
         'cach': 1181,
         'arg': 2306,
         'import': 5519,
         'includ': 6290,
         'miracul': 3,
         'go': 6550,
         'clock_t': 17,
         'random': 1692,
         'unsign': 532,
         'doubl': 1972,
         'iostream': 952,
         'term': 948,
         'rnd': 36,
         '93': 188,
         'algorithm': 869,
         'sum': 830,
         'seem': 9026,
         'similar': 3152,
         'clocks_per_sec': 18,
         'faster': 756,
         'almost': 757,
         'second': 4921,
         'strang': 766,
         'gener': 6115,
         'somewhat': 309,
         'new': 21376,
         'static': 5336,
         'sort': 2245,
         'primari': 1023,
         'process': 3882,
         '54': 423,
         'make': 12498,
         '1000000000': 25,
         '

In [9]:
stop_words = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [10]:
question = """
How would you print out the data in a binary tree, level by level, starting at the top?
"""
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
tokenizer = RegexpTokenizer(r'\w+')

stemmer = PorterStemmer()
tags_freqency_stem = Counter()
tags_doc_freq = Counter()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
question = question.lower()
question_text = BeautifulSoup(question, "lxml").get_text()
tokens = tokenizer.tokenize(question_text)
filtered_sentence = [w for w in tokens if not w in stop_words]

filtered_stems = [stemmer.stem(plural) for plural in filtered_sentence]
filtered_stems_set = set(filtered_stems)
term_frequency = Counter(filtered_stems)
tags_doc_freq += Counter(filtered_stems_set)
tags_freqency_stem += (term_frequency)
    
print(tags_freqency_stem, tags_doc_freq)

Counter({'level': 2, 'print': 1, 'data': 1, 'binari': 1, 'tree': 1, 'start': 1}) Counter({'binari': 1, 'start': 1, 'tree': 1, 'data': 1, 'print': 1, 'level': 1})


In [ ]:
# for each linked question get all the answers and their body. 

query = """
    select parent_id, id, body from `bigquery-public-data.stackoverflow.posts_answers` where parent_id in (SELECT distinct post_id FROM `bigquery-public-data.stackoverflow.post_links`)
    """
query_job = client.query(query)
results = query_job.result()
for row in results:
    file_name = "posts_parent_id_id_body.csv"
    if not os.path.exists(file_name):
        results = query_job.result()
        with open(file_name, "w") as text_file:
            wr = csv.writer(text_file)
            for row in results:
                wr.writerow([row.parent_id] + [row.id] + [row.body])
                
                
                

In [ ]:
# get all the questions with the questions that link to it. 
# get all the related_post_id because we need their titles as text. 

#SELECT distinct related_post_id FROM [bigquery-public-data:stackoverflow.post_links] group by related_post_id

# then iterate over the related_post_id and get all the questions id. and then for each question id find its, title, body, score. 
# similarly for each related_post_id

query = """
    select id, title, body, tags from `bigquery-public-data.stackoverflow.posts_questions` where id in (SELECT distinct post_id FROM `bigquery-public-data.stackoverflow.post_links`)
    """
query_job = client.query(query)
results = query_job.result()
for row in results:
    file_name = "post_id_title_body_tags.csv"
    if not os.path.exists(file_name):
        results = query_job.result()
        with open(file_name, "w") as text_file:
            wr = csv.writer(text_file)
            for row in results:
                wr.writerow([row.id] + [row.title] + [row.body] + [row.tags])
    